__Structural Estimation__ <br>
Problem Set 4 <br>
February 18, 2019 <br>
Ginha Kim <br>

In [109]:
# Import libraries
import numpy as np
import scipy.stats as sts
import pandas as pd
from pandas import DataFrame, Series
import scipy.integrate as integ
import scipy.optimize as opt
import scipy.special as spc
import matplotlib.pyplot as plt
%matplotlib notebook

## Problem 1
Estimating the Brock and Mirman (1972) model by SMM

In [110]:
# Load Data

data = pd.read_csv('NewMacroSeries.txt',sep=',',header=None,names = ['c','k','w','r','y'])

## (a)

In [111]:
# Define primitives

k_1 = data['k'].mean()
T  = 100
S  = 1000
u_sims = np.random.uniform(0,1,(T,S))

In [112]:
# Define function that generates simulated values

def sim_vals(alpha, beta, rho, mu, sigma, u_sims):
    
    # use uniform draws to generate normally distributed errors
    eps = sts.norm.ppf(u_sims, loc = 0, scale = sigma)
    
    # use equation (5) to generate simulated values for z
    z_s = np.zeros((T,S))
    z_0 = mu
    z_s[0,:] = (rho * z_0) + ((1-rho)*mu) + eps[0,:]
    
    for i in range(1,T):
        z_s[i,:] = (rho * z_s[i-1,:]) + ((1-rho)*mu) + eps[i,:]
    
    # use euqtion (7) to generate simulated values for k
    k_s = np.zeros((T+1,S))
    k_s[0,:] = k_1
    
    for i in range(1,T+1):
        k_s[i,:] = alpha * beta * np.exp(z_s[i-1,:])*(k_s[i-1,:]**alpha)
    
    k_st  = k_s[:-1,:]
    k_st1 = k_s[1:,:]
    
    # use equation (3) and (4) to generate simulated values for w and r
    w_s = np.zeros((T,S))
    r_s = np.zeros((T,S))
    
    for i in range(T):
        w_s[i,:] = (1-alpha)*np.exp(z_s[1,:]) * (k_st[i,:]**alpha)
    
    for i in range(T):
        r_s[i,:] = alpha * np.exp(z_s[1,:]) * (k_st[i,:]**(alpha-1))
    
    # use equation (2) to generate simulated values for c
    c_s = np.zeros((T,S))
    
    for i in range(T):
        c_s[i,:] = w_s[i,:] + r_s[i,:]*k_st[i,:] - k_st1[i,:]
    
    # use equation (6) to generate simulated values for y
    y_s = np.zeros((T,S))
    
    for i in range(T):
        y_s[i,:] = np.exp(z_s[i,:]) * (k_st[i,:]**alpha)
        
    
    return c_s, k_st, y_s

In [119]:
# Generate data moments

# moment 1 : mean(c_t)
dm1 = np.mean(data['c'])
# moment 2 : mean(k_t)
dm2 = np.mean(data['k'])
# moment 3 : mean(c_t/y_t)
dm3 = np.mean(data['c']/data['y'])
# moment 4 : var(y_t)
dm4 = np.var(data['y'])
# moment 5 : corr(c_t,c_t-1)
ct  = data['c'][1:100]
ct_1 = data['c'][0:99]
dm5 = np.corrcoef(ct,ct_1)[0][1]
# moment 6 : corr(c_t,k_t)
dm6 = np.corrcoef(data['c'],data['k'])[0][1]

data_moms = np.array([dm1,dm2,dm3,dm4,dm5,dm6])

In [124]:
# Define function that generates model moments

def model_moms(alpha, beta, rho, mu, sigma, u_sims):
    
    c_s, k_st, y_s = sim_vals(alpha, beta, rho, mu, sigma, u_sims)
    mm1 = np.mean(c_s)
    mm2 = np.mean(k_st)
    mm3 = np.mean(c_s/y_s)
    mm4 = np.mean(np.var(y_s,axis=0))
    
    corr_c = np.zeros((1,1000))
    corr_ck = np.zeros((1,1000))
    
    for i in range(1000): 
        corr_c[0,i] = np.corrcoef(c_s[0:99,i],c_s[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c_s[:,i],k_st[:,i])[0,1]
    
    mm5 = np.mean(corr_c)
    mm6 = np.mean(corr_ck)
    
    model_moms = np.array([mm1,mm2,mm3,mm4,mm5,mm6])
    
    return model_moms

In [125]:
print('Data Moments : ')
print(data_moms)
print('Model Moments : ')
print(model_moms(0.5, 0.99, 0.5, 8, 0.5, u_sims))

Data Moments : 
[  9.28179049e+06   6.64398514e+06   5.84200000e-01   2.83778251e+13
   9.40559181e-01   9.40803054e-01]
Model Moments : 
[  2.61430243e+06   3.21344068e+06   8.87436269e-01   4.25761580e+13
   2.91003228e-01   8.43988722e-02]


In [126]:
# Define error vector

def err_vec(alpha, beta, rho, mu, sigma, u_sims):
    
    mms = model_moms(alpha, beta, rho, mu, sigma, u_sims)
    
    return (mms - data_moms)/data_moms

# Define criterion function

def smm_crit(params,*args):
    
    alpha, rho, mu, sigma = params
    
    beta, u_sims = args
    
    err = err_vec(alpha, beta, rho, mu, sigma, u_sims)
    M = np.eye(6)
    crit_val = err.T @ M @ err
    return crit_val

In [129]:
# Simulated Method of Moments

alpha_0 = 0.4916
rho_0   = 0.5018
mu_0    = 8.7595
sigma_0 = 0.2
params_0 = np.array([alpha_0,rho_0,mu_0,sigma_0])

beta    = 0.99

bounds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))

smm_args = (beta,u_sims)
results = opt.minimize(smm_crit,params_0, args = smm_args,method = 'L-BFGS-B', bounds = bounds)

alpha_smm_a, rho_smm_a, mu_smm_a, sigma_smm_a = results.x 

err_smm_a = err_vec(alpha_smm_a, beta, rho_smm_a, mu_smm_a, sigma_smm_a, u_sims)
val_min_crit_smm = err_smm_a.T @ W @ err_smm_a

print(results)
print('alpha smm = ', alpha_smm_a)
print('rho smm = ', rho_smm_a)
print('mu smm = ', mu_smm_a)
print('sigma smm = ', sigma_smm_a)
print('Value of Minimized Criterion Function = ',val_min_crit_smm)

      fun: 1.1119626577574797
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.38378198e-04,   9.14823772e-06,   7.61612995e-06,
         2.71560552e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 205
      nit: 33
   status: 0
  success: True
        x: array([ 0.44554957,  0.78812334,  9.55678217,  0.12415325])
alpha smm =  0.445549569591
rho smm =  0.788123335555
mu smm =  9.55678217316
sigma smm =  0.124153245646
Value of Minimized Criterion Function =  1.11196265776


In [136]:
# Compute Standard Errors

import numpy.linalg as lin

def Jac_err(xvals, usims, alpha, rho, mu, sigma, simple=False):
    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    Jac_err[:, 0] = \
        ((err_vec(alpha + h_alpha, 0.99, rho, mu, sigma, u_sims) -
          err_vec(alpha - h_alpha, 0.99, rho, mu, sigma, u_sims)) / (2 * h_alpha)).flatten()
    Jac_err[:, 1] = \
        ((err_vec(alpha, 0.99, rho + h_rho, mu, sigma, u_sims) -
          err_vec(alpha, 0.99, rho - h_rho, mu, sigma, u_sims)) / (2 * h_rho)).flatten()
    Jac_err[:, 2] = \
        ((err_vec(alpha, 0.99, rho, mu + h_mu, sigma, u_sims) -
          err_vec(alpha, 0.99, rho, mu - h_mu, sigma, u_sims)) / (2 * h_mu)).flatten()
    Jac_err[:, 3] = \
        ((err_vec(alpha, 0.99, rho, mu, sigma + h_sigma, u_sims) -
          err_vec(alpha, 0.99, rho, mu, sigma - h_sigma, u_sims)) / (2 * h_sigma)).flatten()
    
    return Jac_err


S = u_sims.shape[1]
xvals = (data['c'], data['k'], data['w'], data['r'], data['y'])
d_err = Jac_err(xvals, u_sims, alpha_smm_a, rho_smm_a, mu_smm_a, sigma_smm_a, simple = False)
print('Jacobian Matrix = ')
print(d_err)
sig_hat = (1 / S) * lin.inv(d_err.T @ M @ d_err)
print('Variance-Covariance Matrix =')
print(sig_hat)
print('Vector of Moment Differences at Optimum = ')
print(err_smm_a)
print('Standard Error alpha smm = ',np.sqrt(sig_hat[0,0]))
print('Standard Error rho smm = ',np.sqrt(sig_hat[1,1]))
print('Standard Error mu smm = ',np.sqrt(sig_hat[2,2]))
print('Standard Error sigma smm = ',np.sqrt(sig_hat[3,3]))

Jacobian Matrix = 
[[  2.70149865e+01  -8.75523183e-02   1.71790795e+00   3.95330183e-02]
 [  3.54922788e+01   2.90520021e-01   1.96230745e+00   9.12952755e-01]
 [ -1.69462513e+00   1.55558529e-01   9.94716054e-13   9.30098585e-01]
 [  6.13743809e+01   5.25285263e+00   3.47434602e+00   1.79739614e+01]
 [  9.23399095e-01   1.16813127e+00  -6.16874994e-04   9.64811423e-01]
 [ -2.87614160e-01  -1.05104094e+00   9.64905301e-03  -1.38979735e+00]]
Variance-Covariance Matrix =
[[  8.82318157e-05  -1.01361975e-04  -1.50651085e-03   1.97536086e-05]
 [ -1.01361975e-04   9.61511098e-04   1.79235240e-03  -2.86043974e-04]
 [ -1.50651085e-03   1.79235240e-03   2.58872035e-02  -3.87862218e-04]
 [  1.97536086e-05  -2.86043974e-04  -3.87862218e-04   9.56992555e-05]]
Vector of Moment Differences at Optimum = 
[-0.03977885  0.10648236  0.01689979 -0.03823566 -0.61968085 -0.84456491]
Standard Error alpha smm =  0.00939317921159
Standard Error rho smm =  0.0310082424241
Standard Error mu smm =  0.160895007

## (b)

In [142]:
# Define function to generate model moments

def model_moms_b(alpha, beta, rho, mu, sigma, u_sims):
    
    c_s, k_st, y_s = sim_vals(alpha, beta, rho, mu, sigma, u_sims)
    mm1 = np.mean(c_s,0)
    mm2 = np.mean(k_st,0)
    mm3 = np.mean(c_s/y_s,0)
    mm4 = np.mean([np.var(y_s[:,i],axis=0) for i in range(1000)])
    
    corr_c = np.zeros((1,1000))
    corr_ck = np.zeros((1,1000))
    
    for i in range(1000): 
        corr_c[0,i] = np.corrcoef(c_s[0:99,i],c_s[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c_s[:,i],k_st[:,i])[0,1]
    
    mm5 = np.mean(corr_c)
    mm6 = np.mean(corr_ck)
    
    model_moms = np.array([mm1,mm2,mm3,mm4,mm5,mm6])
    
    return model_moms

In [147]:
# Define funtion to construct optimal weighting matrix

def get_err_mat(xvals,alpha,beta,rho,mu,sigma,simple=False):
    
    R = 6 
    S = u_sims.shape[1]
    err_mat = np.zeros((R,S))
    
    c_s, k_st, y_s = sim_vals(alpha, beta, rho, mu, sigma, u_sims)
    
    dm1, dm2, dm3, dm4, dm5, dm6 = data_moms
    mm1, mm2, mm3, mm4, mm5, mm6 = model_moms_b(alpha, beta, rho, mu, sigma, u_sims)

   
    if simple:
        err_mat[0, :] = mm1 - dm1
        err_mat[1, :] = mm2 - dm2
        err_mat[2, :] = mm3 - dm3
        err_mat[3, :] = mm4 - dm4
        err_mat[4, :] = mm5 - dm5
        err_mat[5, :] = mm6 - dm6
        
    else:
        err_mat[0, :] = (mm1 - dm1) / dm1
        err_mat[1, :] = (mm2 - dm2) / dm2
        err_mat[2, :] = (mm3 - dm3) / dm3
        err_mat[3, :] = (mm4 - dm4) / dm4
        err_mat[4, :] = (mm5 - dm5) / dm5
        err_mat[5, :] = (mm6 - dm6) / dm6
    
    return err_mat

In [148]:
# Compute the optimal weighting matrix

xvals   = (data['c'], data['k'], data['w'], data['r'], data['y'])
err_mat = get_err_mat(xvals,alpha_smm_a, 0.99, rho_smm_a, mu_smm_a, sigma_smm_a,simple=False)
VCV_b = (1 / u_sims.shape[1]) * (err_mat @ err_mat.T)
print('Variance-Covariance Matrix of Moment Vector =')
print(VCV_b)
W_2 = lin.inv(VCV_b)
print('Optimal Two-Step Weighting Matrix =')
print(W_2)

Variance-Covariance Matrix of Moment Vector =
[[  7.48253678e-02  -1.12738322e-04   6.88582836e-02   1.52097024e-03
    2.46501886e-02   3.35958170e-02]
 [ -1.12738322e-04   2.50332076e-02  -5.29230914e-03  -4.07142277e-03
   -6.59850778e-02  -8.99312631e-02]
 [  6.88582836e-02  -5.29230914e-03   7.72612503e-02  -6.46174489e-04
   -1.04724752e-02  -1.42729683e-02]
 [  1.52097024e-03  -4.07142277e-03  -6.46174489e-04   1.46196536e-03
    2.36939035e-02   3.22924931e-02]
 [  2.46501886e-02  -6.59850778e-02  -1.04724752e-02   2.36939035e-02
    3.84004352e-01   5.23360699e-01]
 [  3.35958170e-02  -8.99312631e-02  -1.42729683e-02   3.22924931e-02
    5.23360699e-01   7.13289888e-01]]
Optimal Two-Step Weighting Matrix =
[[  5.16270609e+02  -4.16810950e+02  -5.04737930e+02  -2.35151218e+02
   -5.59343042e+01  -3.54707991e+01]
 [ -4.16810950e+02   4.13191471e+02   4.14564575e+02   2.25242324e+02
    5.09316057e+01   3.25555766e+01]
 [ -5.04737930e+02   4.14564575e+02   5.07104856e+02   2.3383

In [149]:
# Define error vector

def err_vec(alpha, beta, rho, mu, sigma, u_sims):
    
    mms = model_moms(alpha, beta, rho, mu, sigma, u_sims)
    
    return (mms - data_moms)/data_moms

# Define criterion function

def smm_critb(params,*args):
    
    alpha, rho, mu, sigma = params
    
    beta, u_sims = args
    
    err = err_vec(alpha, beta, rho, mu, sigma, u_sims)
    
    crit_val = err.T @ W_2 @ err
    
    return crit_val

In [151]:
# Simulated Method of Moments

alpha_0 = 0.4916
rho_0   = 0.5018
mu_0    = 8.7595
sigma_0 = 0.2
params_0 = np.array([alpha_0,rho_0,mu_0,sigma_0])

beta    = 0.99

bounds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))

smm_args = (beta,u_sims)
results = opt.minimize(smm_critb,params_0, args = smm_args,method = 'L-BFGS-B', bounds = bounds)

alpha_smm_b, rho_smm_b, mu_smm_b, sigma_smm_b = results.x 

err_smm_b = err_vec(alpha_smm_b, beta, rho_smm_b, mu_smm_b, sigma_smm_b,u_sims)
val_min_crit_smm_b = err_smm_b.T @ W_2 @ err_smm_b

print(results)
print('alpha smm two-step = ', alpha_smm_b)
print('rho smm two-step = ', rho_smm_b)
print('mu smm two-step = ', mu_smm_b)
print('sigma smm two-step = ', sigma_smm_b)
print('Value of Minimized Criterion Function = ',val_min_crit_smm_b)

      fun: -1621615215982449.8
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.09335810e+16,   2.92213168e+15,   5.55851175e+14,
        -2.25260000e+14])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 545
      nit: 68
   status: 0
  success: True
        x: array([ 0.70553161, -0.88161838,  5.25011179,  0.15090522])
alpha smm two-step =  0.705531613525
rho smm two-step =  -0.881618379936
mu smm two-step =  5.25011179081
sigma smm two-step =  0.1509052225
Value of Minimized Criterion Function =  -1.62161521598e+15


In [152]:
# Compute and Report Standard Errors
d_err_b = Jac_err(xvals, u_sims, alpha_smm_b, rho_smm_b, mu_smm_b, sigma_smm_b, simple = False)
print('Jacobian Matrix = ')
print(d_err)
sig_hat_b = (1 / S) * lin.inv(d_err_b.T @ M @ d_err_b)
print('Variance-Covariance Matrix =')
print(sig_hat_b)
print('Vector of Moment Differences at Optimum = ')
print(err_smm_b)
print('Standard Error alpha smm two-step = ',np.sqrt(sig_hat_b[0,0]))
print('Standard Error rho smm two-step = ',np.sqrt(sig_hat_b[1,1]))
print('Standard Error mu smm two-step = ',np.sqrt(sig_hat_b[2,2]))
print('Standard Error sigma smm two-step = ',np.sqrt(sig_hat_b[3,3]))

Jacobian Matrix = 
[[  2.70149865e+01  -8.75523183e-02   1.71790795e+00   3.95330183e-02]
 [  3.54922788e+01   2.90520021e-01   1.96230745e+00   9.12952755e-01]
 [ -1.69462513e+00   1.55558529e-01   9.94716054e-13   9.30098585e-01]
 [  6.13743809e+01   5.25285263e+00   3.47434602e+00   1.79739614e+01]
 [  9.23399095e-01   1.16813127e+00  -6.16874994e-04   9.64811423e-01]
 [ -2.87614160e-01  -1.05104094e+00   9.64905301e-03  -1.38979735e+00]]
Variance-Covariance Matrix =
[[  7.43969783e-05   1.28460394e-05  -1.33365255e-03   1.76649806e-05]
 [  1.28460394e-05   6.05893045e-04  -2.39845070e-04   2.54342830e-04]
 [ -1.33365255e-03  -2.39845070e-04   2.39225060e-02  -3.29653797e-04]
 [  1.76649806e-05   2.54342830e-04  -3.29653797e-04   1.19873021e-04]]
Vector of Moment Differences at Optimum = 
[-0.20835933  1.45905751 -0.35761979  0.01727652 -1.91103838 -0.150091  ]
Standard Error alpha smm two-step =  0.00862536829991
Standard Error rho smm two-step =  0.024614894778
Standard Error mu s